***Wikipedia article ID<br>
Freebase ID<br>
Book title<br>
Author<br>
Publication date<br>
Book genres (Freebase ID:name tuples)<br>
Plot summary***

***Required libraries***

In [17]:
import os
import pandas as pd
import csv
from tqdm import tqdm
import json
import ast
import requests
from bs4 import BeautifulSoup
import nltk
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [19]:
data = []

with open("archive/booksummaries.txt", 'r') as f:
    reader = csv.reader(f, dialect='excel-tab')
    for row in tqdm(reader):
        data.append(row)


16559it [00:00, 29517.09it/s]


In [20]:
wiki_book_id=[]
book_title=[]
book_author=[]
book_publication_date=[]
book_genre=[]
book_summary=[]
book_cover_image=[]
for i in tqdm(data):
    wiki_book_id.append(i[0])
    book_title.append(i[2])
    book_author.append(i[3])
    book_publication_date.append(i[4])
    book_genre.append(i[5])
    book_summary.append(i[6])

books = pd.DataFrame({'wiki_book_id': wiki_book_id,
                       'book_title': book_title,
                       'book_author':book_author,
                       'book_publication_date':book_publication_date,
                       'book_genre': book_genre,
                        'book_summary': book_summary})


100%|██████████| 16559/16559 [00:00<00:00, 974156.75it/s]


In [21]:
books.head()

,wiki_book_id,book_title,book_author,book_publication_date,book_genre,book_summary
0,620,Animal Farm,George Orwell,1945-08-17,"{""/m/016lj8"": ""Roman \u00e0 clef"", ""/m/06nbt"":...","Old Major, the old boar on the Manor Farm, ca..."
1,843,A Clockwork Orange,Anthony Burgess,1962,"{""/m/06n90"": ""Science Fiction"", ""/m/0l67h"": ""N...","Alex, a teenager living in near-future Englan..."
2,986,The Plague,Albert Camus,1947,"{""/m/02m4t"": ""Existentialism"", ""/m/02xlf"": ""Fi...",The text of The Plague is divided into five p...
3,1756,An Enquiry Concerning Human Understanding,David Hume,,,The argument of the Enquiry proceeds by a ser...
4,2080,A Fire Upon the Deep,Vernor Vinge,,"{""/m/03lrw"": ""Hard science fiction"", ""/m/06n90...",The novel posits that space around the Milky ...


In [22]:
books.shape

(16559, 6)

In [23]:
json.loads(books['book_genre'][0]).values()

dict_values(['Roman à clef', 'Satire', "Children's literature", 'Speculative fiction', 'Fiction'])

In [24]:
def make_genre(obj):
    try:
        return list(json.loads(obj).values())
    except:
        return []


In [25]:
books['book_genre'] = books['book_genre'].apply(make_genre)


In [26]:
books['book_summary'] = books['book_summary'].apply(lambda x:x.split())
books['book_author'] = books['book_author'].apply(lambda x:[x])
books['book_publication_date'] = books['book_publication_date'].apply(lambda x:x.split())


In [27]:
books['book_author'] = books['book_author'].apply(lambda x:[i.replace(' ','') for i in x])

In [28]:
books.head()

,wiki_book_id,book_title,book_author,book_publication_date,book_genre,book_summary
0,620,Animal Farm,[GeorgeOrwell],[1945-08-17],"[Roman à clef, Satire, Children's literature, ...","[Old, Major,, the, old, boar, on, the, Manor, ..."
1,843,A Clockwork Orange,[AnthonyBurgess],[1962],"[Science Fiction, Novella, Speculative fiction...","[Alex,, a, teenager, living, in, near-future, ..."
2,986,The Plague,[AlbertCamus],[1947],"[Existentialism, Fiction, Absurdist fiction, N...","[The, text, of, The, Plague, is, divided, into..."
3,1756,An Enquiry Concerning Human Understanding,[DavidHume],[],[],"[The, argument, of, the, Enquiry, proceeds, by..."
4,2080,A Fire Upon the Deep,[VernorVinge],[],"[Hard science fiction, Science Fiction, Specul...","[The, novel, posits, that, space, around, the,..."


In [29]:
books['tags'] = books['book_author']+books['book_publication_date']+books['book_genre']+books['book_summary']
books.drop(['book_author','book_genre','book_publication_date','book_summary'],axis=1,inplace=True)

In [30]:
books.head()

,wiki_book_id,book_title,tags
0,620,Animal Farm,"[GeorgeOrwell, 1945-08-17, Roman à clef, Satir..."
1,843,A Clockwork Orange,"[AnthonyBurgess, 1962, Science Fiction, Novell..."
2,986,The Plague,"[AlbertCamus, 1947, Existentialism, Fiction, A..."
3,1756,An Enquiry Concerning Human Understanding,"[DavidHume, The, argument, of, the, Enquiry, p..."
4,2080,A Fire Upon the Deep,"[VernorVinge, Hard science fiction, Science Fi..."


In [31]:
books['tags'] = books['tags'].apply(lambda x:" ".join(x))
books['tags'] = books['tags'].apply(lambda x:x.lower())

In [33]:
books.head()

,wiki_book_id,book_title,tags
0,620,Animal Farm,georgeorwell 1945-08-17 roman à clef satire ch...
1,843,A Clockwork Orange,anthonyburgess 1962 science fiction novella sp...
2,986,The Plague,albertcamus 1947 existentialism fiction absurd...
3,1756,An Enquiry Concerning Human Understanding,davidhume the argument of the enquiry proceeds...
4,2080,A Fire Upon the Deep,vernorvinge hard science fiction science ficti...


***Steaming***

In [34]:
ps = PorterStemmer()

def stming_function(obj):
    res=[]
    for i in obj.split():
        res.append(ps.stem(i))
    return " ".join(res)

In [35]:
books['tags'] = books['tags'].apply(stming_function)

In [36]:
books.head()

,wiki_book_id,book_title,tags
0,620,Animal Farm,georgeorwel 1945-08-17 roman à clef satir chil...
1,843,A Clockwork Orange,anthonyburgess 1962 scienc fiction novella spe...
2,986,The Plague,albertcamu 1947 existenti fiction absurdist fi...
3,1756,An Enquiry Concerning Human Understanding,davidhum the argument of the enquiri proce by ...
4,2080,A Fire Upon the Deep,vernorving hard scienc fiction scienc fiction ...


***CountVectorizing***

In [37]:
cvt = CountVectorizer(max_features=2000,stop_words='english')

In [38]:
vectors = cvt.fit_transform(books['tags']).toarray()

In [39]:
cvt.get_feature_names_out()

array(['000', '01', '02', ..., 'young', 'younger', 'youth'], dtype=object)

In [40]:
similarity = cosine_similarity(vectors)

In [41]:
books['book_title'][121]

"Winter's Heart"

In [42]:
def recomend(movie):
    book_idx = books[books['book_title'] == movie].index[0]
    #index gives obj and index[0] gives index of row ie number of row
    distance = similarity[book_idx]
    #distance contain the similarity of that movie with other movies
    recomendations = sorted(list(enumerate(distance)),reverse=True,key = lambda x:x[1])[1:11]
    #list of top 10 similar movies 0 th movie is same as main movie name
    for i in recomendations:
        print(books['book_title'][i[0]])

In [43]:
recomend("Harry Potter and the Philosopher's Stone")

Harry Potter and the Order of the Phoenix
Harry Potter and the Goblet of Fire
Harry Potter and the Chamber of Secrets
Pronto
Harry Potter and the Prisoner of Azkaban
Harry Potter and the Deathly Hallows
Harry Potter and the Half-Blood Prince
Ghost Story
The Devil's Star
Necroscope V: Deadspawn


In [44]:
pickle.dump(books.to_dict(),open('books.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))